# 飞书机器人上手教程 (3) — AI 对话集成

本 Notebook 演示如何将飞书机器人接入 LLM（大语言模型），让每个用户都有独立的对话记忆。

## 核心组件

- **`LarkBot`**：飞书机器人，负责收发消息
- **`ChatSession`**：会话管理器，每个用户维护独立的对话历史
- **`chattool.Chat`**：底层 LLM 对话接口（支持 OpenAI / Azure / 其他兼容接口）

## 前置条件

- 飞书凭证：`FEISHU_APP_ID` / `FEISHU_APP_SECRET`
- LLM 凭证：`OPENAI_API_KEY`（或其他兼容接口配置）

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from chattool.tools.lark import LarkBot, ChatSession

TEST_USER = "rexwzh"
ID_TYPE   = "user_id"

## 1. ChatSession 基本用法

`ChatSession` 对 `chattool.Chat` 做了薄封装，核心是 **每个 `user_id` 对应独立的 Chat 实例**。

In [ ]:
# 基本用法
session = ChatSession(system="你是一个简洁的工作助手，用中文回答，每次不超过50字。")

# 模拟两个用户各自对话
reply1 = session.chat("rexwzh", "你好，介绍一下你自己")
reply2 = session.chat("alice", "用一句话解释什么是 Python")

print(f"rexwzh 的回复: {reply1}")
print(f"alice 的回复:  {reply2}")
print(f"\n当前活跃会话数: {session.user_count()}")

In [ ]:
# 多轮对话 —— 同一用户的历史会自动保留
session.chat("rexwzh", "给我列3个 Python 异步框架")
reply = session.chat("rexwzh", "你刚才说的第一个框架，能展开介绍吗？")  # 依赖上下文
print(reply)

In [ ]:
# 清除某用户的对话历史
session.clear("rexwzh")
print(f"清除后活跃会话数: {session.user_count()}")

# 再次提问，上下文已被清除
reply = session.chat("rexwzh", "你刚才说的框架是什么？")
print(f"清除后回复: {reply}")  # 应该回答不知道之前说了什么

## 2. 限制对话轮数（防止 token 膨胀）

`max_history=N` 表示最多保留最近 N 轮对话（每轮 = 1条用户消息 + 1条助手回复）。

In [ ]:
# 只保留最近 3 轮对话
limited_session = ChatSession(
    system="你是一个助手",
    max_history=3
)

for i in range(6):
    limited_session.chat("rexwzh", f"第 {i+1} 条消息")

# 内部 chat.chat 的长度应该是：1(system) + 3轮*2 = 7 条
chat_obj = limited_session._sessions.get("rexwzh")
if chat_obj:
    print(f"历史消息条数: {len(chat_obj.chat)} （期望 ≤ 7）")

## 3. 完整 AI 对话机器人

将 `ChatSession` 与 `LarkBot` 结合，实现完整的对话机器人：

In [ ]:
bot = LarkBot()
session = ChatSession(
    system="你是一个飞书工作助手，回答简洁专业，用中文。",
    max_history=10,
)

# ── 指令处理 ──────────────────────────────────────────
@bot.command("/clear")
def on_clear(ctx):
    """清除当前用户的对话历史"""
    session.clear(ctx.sender_id)
    ctx.reply("✅ 对话记忆已清除，我们重新开始吧！")

@bot.command("/help")
def on_help(ctx):
    ctx.reply(
        "🤖 **AI 助手** 可用指令：\n"
        "  /help   — 显示帮助\n"
        "  /clear  — 清除对话历史\n"
        "  /status — 查看当前状态\n\n"
        "直接发消息即可开始对话！"
    )

@bot.command("/status")
def on_status(ctx):
    count = session.user_count()
    has = session.has_session(ctx.sender_id)
    ctx.reply(
        f"📊 系统状态\n"
        f"  活跃会话数：{count}\n"
        f"  你的会话：{'已建立' if has else '未开始'}"
    )

# ── 核心 AI 对话 ──────────────────────────────────────
@bot.on_message
def on_chat(ctx):
    if not ctx.text.strip():
        return  # 忽略空消息
    try:
        reply = session.chat(ctx.sender_id, ctx.text)
        ctx.reply(reply)
    except Exception as e:
        ctx.reply(f"⚠️ 出错了：{e}")

print("✅ AI 对话机器人配置完成")
print("指令列表: /help  /clear  /status")
print("调用 bot.start_background() 启动")

### 验证：模拟一轮对话（不需要真实网络）

In [ ]:
import json
from unittest.mock import MagicMock

def make_event(text, user_id="rexwzh"):
    msg = MagicMock()
    msg.message_type = "text"
    msg.content = json.dumps({"text": text})
    msg.message_id = "om_test"; msg.chat_id = "p2p_test"
    msg.chat_type = "p2p"; msg.thread_id = None
    sid = MagicMock(); sid.open_id = user_id
    sender = MagicMock(); sender.sender_id = sid; sender.sender_type = "user"
    event = MagicMock(); event.message = msg; event.sender = sender
    data = MagicMock(); data.event = event
    return data

# 模拟 /help 指令
bot._dispatch_message(make_event("/help"))
print("✅ /help 指令分发成功")

# 模拟 /clear
session.chat("rexwzh", "预先建立一个会话")
assert session.has_session("rexwzh")
bot._dispatch_message(make_event("/clear"))
assert not session.has_session("rexwzh")
print("✅ /clear 清除了 rexwzh 的会话")

## 4. 真实启动 & 发消息验证

In [ ]:
# 主动给 rexwzh 发一条欢迎消息
bot_raw = LarkBot()
resp = bot_raw.send_text(TEST_USER, ID_TYPE,
    "🤖 AI 助手已上线！\n"
    "直接发消息开始对话，或发送 /help 查看指令。"
)
print("发送状态:", "✅" if resp.success() else f"❌ {resp.msg}")

In [ ]:
# 启动机器人（后台 WebSocket 长连接）
bot.start_background()
print("🚀 AI 对话机器人已启动！")
print("向机器人发消息，它会用 AI 回复。")
print("发送 /clear 可以重置对话历史。")

## 5. 自定义 System Prompt 场景

通过修改 `system` 参数，可以快速定制不同场景的助手：

In [ ]:
# 场景 1：代码审查助手
code_session = ChatSession(
    system="你是一个代码审查专家，专注 Python。指出代码问题并给出改进建议，回答简洁。"
)

# 场景 2：HR 问答助手  
hr_session = ChatSession(
    system="你是公司 HR 助手，回答员工关于假期、薪资、福利的问题。友好专业，不确定时建议联系 HR 部门。"
)

# 场景 3：技术文档助手（带文档上下文）
doc_context = """
以下是产品文档摘要：
- 飞书机器人支持文本、富文本、卡片三种消息类型
- 开发者需要申请 App ID 和 App Secret
- 支持 Webhook 和 WebSocket 两种接收消息模式
"""
doc_session = ChatSession(
    system=f"你是产品文档助手，基于以下文档回答问题：\n{doc_context}"
)

print("三种 Session 场景配置完成：")
print("  code_session — 代码审查")
print("  hr_session   — HR 问答")
print("  doc_session  — 文档问答")

## 小结

| 操作 | 代码 |
|------|------|
| 创建会话管理器 | `session = ChatSession(system="...")` |
| 对话（自动管理历史） | `reply = session.chat(user_id, text)` |
| 限制历史轮数 | `ChatSession(max_history=10)` |
| 清除某用户历史 | `session.clear(user_id)` |
| 清空所有会话 | `session.clear_all()` |
| 查询活跃用户数 | `session.user_count()` |

下一篇：[04 — 交互卡片](lark_04_cards.ipynb)